In [1]:
# An attempt to make Neural net w/ more than 1 hidden layer

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import time

In [3]:
# 785 cols bc of labels row - only 28*28 = 784 pixels
data = pd.read_csv('mnist_train.csv')
data_test = pd.read_csv('mnist_test.csv')

data = np.array(data)
data_test = np.array(data_test)

data = data.T
data_test = data_test.T

def preprocess_data(X):
    """Preprocess the input data"""
    # Scale pixels to [0, 1] range
    X = X / 255.0
    return X

In [4]:
x_train = data[1:data[0].size].T # 2D Array: each row (array) is a picture
x_test = data_test[1:data[0].size].T # 2D Array: each row (array) is a picture
x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)
x_labels = data[0] # Array: each number is the correct label for the corresponding index in x_train
x_test_labels = data_test[0]
print(x_train.shape)
print(x_test_labels)
print(x_test)

(59999, 784)
[7 2 1 ... 4 5 6]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
# Initialize parameters for every layer
# Weights & biases
# 3 hidden layers w/ 32, 32, 16 neurons respectively
def init_params():
    W_1 = np.random.randn(32, 784) * np.sqrt(2.0/784) # Edges from layer 1 to layer 2
    B_1 = np.zeros(32) # Biases for layer 2

    W_2 = np.random.randn(32, 32) * np.sqrt(2.0/32) # Edges from layer 2 to layer 3
    B_2 = np.zeros(32) # Biases for layer 3

    W_3 = np.random.randn(32, 32) * np.sqrt(2.0/32) # Edges from layer 3 to layer 4
    B_3 = np.zeros(32) # Biases for layer 4
    
    W_4 = np.random.randn(10, 32) * np.sqrt(2.0/16) # Edges from layer 4 to layer 5
    B_4 = np.zeros(10) # Biases for layer 5
    return W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4

In [6]:
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    # Ensure input is at least 1D
    Z = np.atleast_1d(Z)
    
    # Shift values by max for numerical stability
    Z_shifted = Z - np.max(Z, axis=-1, keepdims=True)
    
    # Avoid overflow
    exp_Z = np.exp(np.clip(Z_shifted, -709, 709))  # np.log(np.finfo(np.float64).max) ≈ 709
    
    # Avoid division by zero
    sum_exp_Z = np.sum(exp_Z, axis=-1, keepdims=True)
    sum_exp_Z = np.maximum(sum_exp_Z, np.finfo(float).tiny)
    
    softmax_output = exp_Z / sum_exp_Z
    
    # Ensure probabilities sum to 1
    softmax_output = softmax_output / np.sum(softmax_output, axis=-1, keepdims=True)
    
    return softmax_output

# Cross-entropy cost function
def cross_entropy_loss(y_true, y_pred):
    # y_true is a one-hot encoded vector (e.g., [0, 1, 0, ...])
    # y_pred is the softmax output
    epsilon = 1e-10  # To avoid log(0)
    return -np.sum(y_true * np.log(y_pred + epsilon))  # Sum over classes

# Cost function
def get_cost(Z_2, i):
    error_arr = np.zeros(10)
    error_arr[x_labels[i]] = 1 # Corresponding to one_picture
    cost = cross_entropy_loss(error_arr, Z_2)
    return cost, error_arr


In [7]:
# Forward Propagation
# Z_1: neurons in hidden layer 1
# Z_2: neurons in hidden layer 2
# Z_3: neurons in hidden layer 3
# Z_4: neurons in output layer

def forward_prop(i, W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4):
    one_picture = x_train[i] # A single sample picture
    
    Z_1 = W_1.dot(one_picture) + B_1
    Z_1 = ReLU(Z_1)

    Z_2 = W_2.dot(Z_1) + B_2
    Z_2 = ReLU(Z_2)

    Z_3 = W_3.dot(Z_2) + B_3
    Z_3 = ReLU(Z_3)
    
    Z_4 = W_4.dot(Z_3) + B_4
    Z_4 = softmax(Z_4)


    
    return Z_1, Z_2, Z_3, Z_4

In [8]:
# Backpropagation
def back_prop(Z_1, Z_2, W_2, Z_3, W_3, Z_4, W_4, i):
    one_picture = x_train[i]
    error_arr = get_cost(Z_4, i)[1]

    dZ_4 = Z_4 - error_arr
    dW_4 = np.outer(dZ_4, Z_3)
    dB_4 = dZ_4

    dZ_3 = W_4.T.dot(dZ_4)
    dZ_3 *= (Z_3 > 0)
    dW_3 = np.outer(dZ_3, Z_2)
    dB_3 = dZ_3

    dZ_2 = W_3.T.dot(dZ_3)
    dZ_2 *= (Z_2 > 0)
    dW_2 = np.outer(dZ_2, Z_1)
    dB_2 = dZ_2

    dZ_1 = W_2.T.dot(dZ_2)
    dZ_1 *= (Z_1 > 0)
    dW_1 = np.outer(dZ_1, one_picture)
    dB_1 = dZ_1

    return dW_1, dB_1, dZ_1, dW_2, dB_2, dZ_2, dW_3, dB_3, dZ_3, dW_4, dB_4, dZ_4
    """
    one_picture = x_train[i]
    error_arr = get_cost(Z_2, i)[1]
    
    dZ_2 = Z_2 - error_arr
    dW_2 = np.outer(dZ_2, Z_1)
    dB_2 = dZ_2
    
    dZ_1 = W_2.T.dot(dZ_2)
    dZ_1 *= (Z_1 > 0)
    dW_1 = np.outer(dZ_1, one_picture)
    dB_1 = dZ_1
    
    return dW_1, dB_1, dZ_1, dW_2, dB_2, dZ_2
    """

In [9]:
# Update parameters
def update_params(W_4, B_4, W_3, B_3, W_2, B_2, W_1, B_1, dW_4, dB_4, dW_3, dB_3, dW_2, dB_2, dW_1, dB_1, learning_rate):

    W_4 -= dW_4 * learning_rate
    B_4 -= dB_4 * learning_rate
    W_3 -= dW_3 * learning_rate
    B_3 -= dB_3 * learning_rate
    
    W_2 -= dW_2 * learning_rate
    B_2 -= dB_2 * learning_rate
    W_1 -= dW_1 * learning_rate
    B_1 -= dB_1 * learning_rate
    return W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4

In [10]:
def train(epochs, learning_rate):
    train_start = time.time()
    W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4 = init_params()
    leng = x_train.shape[0]
    
    for i in range(epochs):
        #np.random.shuffle(x_train)

        #learning_rate *= 0.9 # Varying learning rate    
        
        iteration = 0
        for j in range(leng):
            iteration = j
            Z_1, Z_2, Z_3, Z_4 = forward_prop(j, W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4)
            dW_1, dB_1, dZ_1, dW_2, dB_2, dZ_2, dW_3, dB_3, dZ_3, dW_4, dB_4, dZ_4 = back_prop(Z_1, Z_2, W_2, Z_3, W_3, Z_4, W_4, j)
            W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4 = update_params(W_4, B_4, W_3, B_3, W_2, B_2, W_1, B_1, dW_4, dB_4, dW_3, dB_3, dW_2, dB_2, dW_1, dB_1, learning_rate)    
        if i % 1 == 0:
            print(f"Epoch {i}: cost = {get_cost(Z_4, iteration)[0]}, LR = {learning_rate}")
    train_end = time.time()
    print(f"Total training time: {(train_end-train_start)} seconds")
    return W_1, B_1, W_2, B_2, W_3, B_3,  W_4, B_4

In [11]:
def predict(X, W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4):
    """
    Make predictions for input images
    
    Parameters:
    X: numpy array of shape (n_samples, 784) - input images
    Returns: tuple (predictions, probabilities)
    - predictions: predicted digit for each image
    - probabilities: softmax probabilities for each digit
    """
    # Ensure input is properly scaled
    X = X / 255.0 if X.max() > 1 else X
    
    # Forward pass
    Z_1 = X.dot(W_1.T) + B_1
    A_1 = np.maximum(0, Z_1)  # ReLU

    Z_2 = A_1.dot(W_2.T) + B_2
    A_2 = np.maximum(0, Z_2)

    Z_3 = A_2.dot(W_3.T) + B_3
    A_3 = np.maximum(0, Z_3)
    
    Z_4 = A_3.dot(W_4.T) + B_4
    probabilities = softmax(Z_4)
    
    # Get predicted digit (class with highest probability)
    predictions = np.argmax(probabilities, axis=1)
    
    return predictions, probabilities

        

In [12]:
W_1, B_1, W_2, B_2, W_3, B_3,  W_4, B_4 = train(5, 0.001)

Epoch 0: cost = 0.022025796551268677, LR = 0.001
Epoch 1: cost = 0.025557602592742215, LR = 0.001
Epoch 2: cost = 0.024190802963432678, LR = 0.001
Epoch 3: cost = 0.011633161907439163, LR = 0.001
Epoch 4: cost = 0.007933217366395006, LR = 0.001
Total training time: 22.28968381881714 seconds


In [13]:
sample_size = 10000
X_sample = x_test[:sample_size]
X_sample = x_test[:sample_size]

inf_start = time.time()

predictions, probabilities = predict(X_sample, W_1, B_1, W_2, B_2, W_3, B_3, W_4, B_4)

correct = 0
for i in range(sample_size):
    correct += (predictions[i] == x_test_labels[i])

inf_end = time.time()

print(f"{correct/sample_size * 100}% correct" )
print(f"{correct}/{sample_size} correct" )
print(f"Time elapsed for inference: {(inf_end-inf_start)} seconds")

96.16% correct
9616/10000 correct
Time elapsed for inference: 0.06954383850097656 seconds
